In [1]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):

        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
spark

In [4]:
import random
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit
from pyspark.sql.types import StringType

In [5]:
import pandas as pd

In [6]:
df =pd.read_csv("reviews.csv", sep='\t')
df = df[['review_id', 'app_id', 'review_text', 'label']]
df.to_csv('reviews.csv', sep='\t', index=False)
df.shape

(32041, 4)

In [7]:
df.groupby('label').count()

,review_id,app_id,review_text
label,,,
0.0,6381,6381,6380
1.0,25659,25659,25615


In [8]:
df.label.mean()

0.8008426966292135

# build model

In [38]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, HashingTF
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import expr
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.feature import Word2Vec
from pyspark.ml import Pipeline
from pyspark.sql.functions import split
from pyspark.sql.types import StructType, StructField, StringType, DoubleType

file_location="reviews.csv"
#text_df = spark.read.text(file_location)

In [10]:
# create a SparkSession 
spark = SparkSession.builder \
    .appName("assignment 3") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.cores", "2") \
    .getOrCreate()

23/05/18 19:21:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
#text_df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine", "true").csv(file_location)

# read csv

text_df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("multiline", "true") \
    .option("delimiter", "\t") \
    .csv(file_location)

text_df = text_df.select(col('review_text'), col('label'))
text_df = text_df.dropna()
text_df.show()

+--------------------+-----+
|         review_text|label|
+--------------------+-----+
|i can confirm tha...|  1.0|
|Really good game,...|  1.0|
|Its not finished ...|  1.0|
|Hey. It's really ...|  1.0|
|          Fun so far|  1.0|
|I'd rather play W...|  0.0|
|I have been playi...|  1.0|
|Nice game! Loads ...|  1.0|
|All hail NA serve...|  1.0|
|enjoying it so fa...|  1.0|
|This game came as...|  1.0|
|         Great fun  |  1.0|
|Fantastic consept...|  1.0|
|You know the game...|  1.0|
|EDIT: My issues h...|  1.0|
|AMAZIIIIIIIIIIING...|  1.0|
|In its current st...|  0.0|
|Играю в Калибр го...|  1.0|
|Secret Word: Prou...|  1.0|
|Fated Word: Death...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [13]:
# Get the number of rows and columns in the DataFrame
num_rows = text_df.count()
num_cols = len(text_df.columns)

# Print the shape of the DataFrame
print("Shape of the DataFrame: (%d, %d)" % (num_rows, num_cols))

Shape of the DataFrame: (31157, 2)


In [14]:
# Remove duplicate rows
text_df = text_df.dropDuplicates()
# Remove rows with missing values
text_df = text_df.na.drop()

In [15]:
text_df.select("label").distinct().show()

+--------------------+
|               label|
+--------------------+
|                 1.0|
|                 0.0|
|It's been at leas...|
+--------------------+



In [16]:
# Remove rows with unexpected labels
text_df = text_df.filter((col("label") == 1.0) | (col("label") == 0.0))
print("Number of rows after filtering: ", text_df.count())

Number of rows after filtering:  28256


In [18]:
train_data, test_data = text_df.limit(10000).randomSplit([0.8, 0.2], seed=7)
train_data.show(5)

+--------------------+-----+
|         review_text|label|
+--------------------+-----+
|\n\n\n\n\n boring...|  0.0|
|\n\nEven though I...|  1.0|
|\nUbisoft began m...|  1.0|
|   As a big fan o...|  1.0|
|  Cyberconnect2 f...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [44]:
# preprocess the data
tokenizer = Tokenizer(inputCol="review_text", outputCol="words")
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", locale="en_US")
count_vectorizer = CountVectorizer(inputCol="filtered_words", outputCol="raw_features")
idf = IDF(inputCol="raw_features", outputCol="features")
string_indexer = StringIndexer(inputCol="label", outputCol="label_index")

# create model
lr = LogisticRegression(featuresCol="features", labelCol="label_index")

23/05/18 20:59:35 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [45]:
# define params grid
param_grid = ParamGridBuilder() \
   .addGrid(count_vectorizer.vocabSize, [1000, 5000]) \
   .addGrid(lr.regParam, [0.01, 0.1]) \
   .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
   .build()

# define the evaluator
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction")

In [46]:
# create pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, count_vectorizer, idf, string_indexer, lr])

# define the cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# fit pipeline to the training data
cv_model = cv.fit(train_data)

# make predictions on the test data
predictions = cv_model.transform(test_data)

23/05/18 20:59:37 WARN CacheManager: Asked to cache already cached data.
23/05/18 20:59:37 WARN CacheManager: Asked to cache already cached data.
23/05/18 20:59:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 20:59:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 20:59:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 20:59:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23

23/05/18 21:00:00 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:00 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:01 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:00:02 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:00:06 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:07 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:07 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:07 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:10 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_U

23/05/18 21:00:27 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:28 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:00:28 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


...

23/05/18 21:01:35 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:01:37 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:01:39 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:01:42 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:01:43 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:01:48 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:01:49 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:01:52 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:01:53 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:01:53 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_U

In [47]:
evaluator = MulticlassClassificationEvaluator(labelCol="label_index", predictionCol="prediction")

# Calculate accuracy
accuracy = evaluator.evaluate(predictions, {evaluator.metricName: "accuracy"})

# Calculate F1 score
f1_score = evaluator.evaluate(predictions, {evaluator.metricName: "f1"})

# Calculate recall
recall = evaluator.evaluate(predictions, {evaluator.metricName: "weightedRecall"})

# Calculate precision
precision = evaluator.evaluate(predictions, {evaluator.metricName: "weightedPrecision"})

print("Accuracy: {:.4f}".format(accuracy))
print("F1 score: {:.4f}".format(f1_score))
print("Recall: {:.4f}".format(recall))
print("Precision: {:.4f}".format(precision))


Accuracy: 0.8673
F1 score: 0.8591
Recall: 0.8673
Precision: 0.8582


In [62]:
from pyspark.mllib.evaluation import MulticlassMetrics

# Convert the predictions and labels to an RDD
predictionAndLabels = predictions.select("prediction", "label_index").rdd.map(lambda r: (r[0], r[1]))

# Instantiate a MulticlassMetrics object
metrics = MulticlassMetrics(predictionAndLabels)

# Get the confusion matrix as a NumPy array
confusion_matrix = metrics.confusionMatrix().toArray()

# Print the confusion matrix
print("Confusion matrix:")
print(confusion_matrix)

/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Confusion matrix:
[[1555.   82.]
 [ 188.  209.]]


In [53]:
# Get the best model from the cross-validation process
best_model = cv_model.bestModel

# Save the my_model
best_model.save("my_logistic_regression")


In [63]:
globals()['models_loaded'] = False
globals()['my_model'] = None

# Toy predict function that returns a random probability. Normally you'd use your loaded globals()['my_model'] here
def predict(df):
    return random.random()

predict_udf = udf(predict, StringType())

def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert to data frame
    df = spark.read.json(rdd)
    df.show()
    
    # Utilize our predict function
    df_withpreds = df.withColumn("pred", predict_udf(
        struct([df[x] for x in df.columns])
    ))
    df_withpreds.show()
    
    # Normally, you wouldn't use a UDF (User Defined Function) Python function to predict as we did here (you can)
    # but an MLlib model you've built and saved with Spark
    # In this case, you need to prevent loading your model in every call to "process" as follows:
    
    # Load in the model if not yet loaded:
    if not globals()['models_loaded']:
        # load in your models here
        globals()['my_model'] = PipelineModel.load('my_logistic_regression')
        globals()['models_loaded'] = True
        
    # And then predict using the loaded model (uncomment below):
    
    df_result = globals()['my_model'].transform(df)
    df_result.show()

In [64]:
ssc = StreamingContext(sc, 10)

In [65]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [66]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

Exception in thread Thread-32:
Traceback (most recent call last):
  File "/Users/anhphuong/opt/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/var/folders/wz/ss5c6j393vv1vr0xc8yky_680000gn/T/ipykernel_64182/1553902016.py", line 11, in run
  File "/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/pyspark/streaming/context.py", line 214, in start
    self._jssc.start()
  File "/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1321, in __call__
  File "/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/pyspark/sql/utils.py", line 190, in deco
    return f(*a, **kw)
  File "/Users/anhphuong/Documents/KUL/Advanced Analytics/spark/spark-3.3.2-bin-hadoop2/python/lib/py4j-0.10.9.5-src.zip/py4j/protocol.py", line 326, in get_return_value
py4j.protocol.Py4JJavaError: An error occurred while

23/05/18 21:19:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
23/05/18 21:19:54 WARN BlockManager: Block input-0-1684419594000 replicated to only 0 peer(s) instead of 1 peers


========= 2023-05-18 21:20:00 =========
+-------+-----+---------+--------------------+
| app_id|label|review_id|         review_text|
+-------+-----+---------+--------------------+
|1651490|    1|138485048|Tiny Life is a gr...|
+-------+-----+---------+--------------------+



+-------+-----+---------+--------------------+-------------------+
| app_id|label|review_id|         review_text|               pred|
+-------+-----+---------+--------------------+-------------------+
|1651490|    1|138485048|Tiny Life is a gr...|0.47672284074442306|
+-------+-----+---------+--------------------+-------------------+

23/05/18 21:20:03 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.
23/05/18 21:20:05 WARN StopWordsRemover: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


23/05/18 21:20:06 ERROR Executor: Exception in task 0.0 in stage 25078.0 (TID 14713)
org.apache.spark.SparkException: Failed to execute user defined function (StringIndexerModel$$Lambda$4148/0x00000008018b1040: (string) => double)
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:190)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:760)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$getByteArrayRdd$1(SparkPlan.scala:364)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:890)
	at org.apache.spark.rdd.RDD.$anonf

...

23/05/19 09:42:04 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2066)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

23/05/19 09:42:14 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:593)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$1(BlockManagerMasterEndpoint.scala:592)
	at org.apache.spar

23/05/19 09:42:24 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:301)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:103)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:87)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:643)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1057)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:238)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.util.Utils$.logUncaughtExceptions(Utils.scala:2066)
	at org.apache.spark.Heartbeater$$anon$1.run(Heartbeater.scala:46)
	at java.base/j

23/05/19 09:42:25 ERROR Executor: Exit as unable to send heartbeats to driver more than 60 times
23/05/19 09:42:35 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.base/java.net.SocketInputStream.socketRead0(Native Method)
	at java.base/java.net.SocketInputStream.socketRead(SocketInputStream.java:115)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:168)
	at java.base/java.net.SocketInputStream.read(SocketInputStream.java:140)
	at java.base/sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at java.base/sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at java.base/sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.base/java.io.InputStreamReader.read(InputStreamReader.java:181)
	at java.base/java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:326)
	at java.base/java.io.BufferedReader.readLine(BufferedReader.java:392)
	at org.apac

Exception in thread "receiver-supervisor-future-0" java.lang.InterruptedException: sleep interrupted
	at java.base/java.lang.Thread.sleep(Native Method)
	at org.apache.spark.streaming.receiver.ReceiverSupervisor.$anonfun$restartReceiver$1(ReceiverSupervisor.scala:196)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at scala.concurrent.Future$.$anonfun$apply$1(Future.scala:659)
	at scala.util.Success.$anonfun$map$1(Try.scala:255)
	at scala.util.Success.map(Try.scala:213)
	at scala.concurrent.Future.$anonfun$map$1(Future.scala:292)
	at scala.concurrent.impl.Promise.liftedTree1$1(Promise.scala:33)
	at scala.concurrent.impl.Promise.$anonfun$transform$1(Promise.scala:33)
	at scala.concurrent.impl.CallbackRunnable.run(Promise.scala:64)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1128)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:628)
	at java.base/java.lang.Thread.run(Thread.ja